In [3]:
# Import necessary libraries
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html
from dash import dash_table
from dash.dependencies import Input, Output
import pandas as pd
from crudModule import AnimalShelter


# MongoDB Connection
username = "root"
password = "6Y1LFaNnsx"
shelter = AnimalShelter(username, password)

# Retrieve data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)

# Initialize the Dash app
app = JupyterDash('ProjectTwoDashboard')



# Define breed filters
water_rescue_breeds = ['Labrador Retriever', 'Newfoundland']
mountain_rescue_breeds = ['Bernese Mountain Dog', 'German Shepherd']
disaster_rescue_breeds = ['Bloodhound', 'Belgian Malinois']


# App layout
app.layout = html.Div([
    html.Div(
        html.Img(src='Grazioso_Salvare_Logo.png', style={'width': '200px', 'display': 'block', 'margin': '0 auto'}),
        style={'textAlign': 'center'}
    ),
    html.H1('Grazioso Salvare Dashboard - Besker Telisma'),
    html.Hr(),
    dcc.RadioItems(
        id='filter-options',
        options=[
            {'label': 'RESET', 'value': 'RESET'},
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
        ],
        value='RESET',
        labelStyle={'display': 'inline-block'}
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_data={'whiteSpace': 'normal', 'height': 'auto'},
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
    ),
    html.Br(),
    html.Div(id='map-id', className='col s12 m6')
])

# Callback for updating data table based on filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-options', 'value')]
)
def update_table(selected_filter):
    if selected_filter == 'RESET':
        filtered_df = df
    elif selected_filter == 'Water Rescue':
        filtered_df = df[df['breed'].isin(water_rescue_breeds)]
    elif selected_filter == 'Mountain or Wilderness Rescue':
        filtered_df = df[df['breed'].isin(mountain_rescue_breeds)]
    elif selected_filter == 'Disaster or Individual Tracking':
        filtered_df = df[df['breed'].isin(disaster_rescue_breeds)]
    else:
        filtered_df = df[df['breed'].str.contains(selected_filter, case=False, na=False)]
    return filtered_df.to_dict('records')

# Callback for updating map based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if index is None or len(index) == 0:
        index = [0]
    
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    # Ensure the columns for latitude and longitude are available
    if 'location_lat' in dff.columns and 'location_long' in dff.columns:
        lat_col = 'location_lat'
        long_col = 'location_long'
    else:
        # Assuming columns 13 and 14 are latitude and longitude if not named
        lat_col = dff.columns[13]
        long_col = dff.columns[14]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row][lat_col], dff.iloc[row][long_col]],
                      children=[
                          dl.Tooltip(dff.iloc[row]['breed']),
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(dff.iloc[row]['name'])
                          ])
                      ])
        ])
    ]

# Run the app
app.run_server(debug=True)

Dash app running on http://127.0.0.1:17596/
